# TP3 - Views, Updates and Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP2.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3

In [2]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
RBA:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
achats:
  0: NB(NUM)
  1: NV(NUM)
  2: LIEU(TEXT)
  3: DATES(NUM)
  4: QTE(NUM)
acheteurs:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
lieu:
  0: LIEU(TEXT)
  1: REGION(TEXT)
producteur:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
produits:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
vins1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
vins2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


Again, we will use **`%%sql`** magic for our queries

In [4]:
%load_ext sql
%sql sqlite:///wine.db

u'Connected: @wine.db'

Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP2

In [5]:
%%sql 
DROP TABLE IF EXISTS producteur;
CREATE TABLE producteur AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1 WHERE NP IS NOT NULL;

DROP TABLE IF EXISTS vins1;
CREATE TABLE vins1 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1 WHERE NV IS NOT NULL;

DROP TABLE IF EXISTS produits;
CREATE TABLE produits AS
SELECT NV, NP, QTE
FROM MASTER1 WHERE NV IS NOT NULL AND NP IS NOT NULL;

DROP TABLE IF EXISTS acheteurs;
CREATE TABLE acheteurs AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM MASTER2 WHERE NB IS NOT NULL;

DROP TABLE IF EXISTS vins2;
CREATE TABLE vins2 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2 WHERE NV IS NOT NULL;

DROP TABLE IF EXISTS lieu;
CREATE TABLE lieu AS
SELECT DISTINCT LIEU, REGION
FROM MASTER2 WHERE LIEU IS NOT NULL;

DROP TABLE IF EXISTS achats;
CREATE TABLE achats AS
SELECT NB, NV, LIEU, DATES, QTE
FROM MASTER2 WHERE NB IS NOT NULL AND NV IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [6]:
%%sql
DROP VIEW IF EXISTS bons_buveurs;
CREATE VIEW bons_buveurs AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM acheteurs WHERE TYPE="moyen" or TYPE="gros";


 * sqlite:///wine.db
Done.
Done.


[]

In [7]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb;

 * sqlite:///wine.db
Done.


[(2, u'Artaud', u'Antonin', u'moyen'),
 (3, u'Aron', u'Raymond', u'gros'),
 (4, u'Apollinaire', u'Guillaume', u'moyen'),
 (6, u'Arrabal', u'Fernando', u'gros'),
 (7, u'Anouilh', u'Jean', u'moyen'),
 (8, u'Aragon', u'Louis', u'gros'),
 (10, u'Andersen', u'Yann', u'gros'),
 (12, u'Bataille', u'Georges', u'moyen'),
 (13, u'Barthes', u'Roland', u'moyen'),
 (14, u'Bory', u'Jean Louis', u'gros'),
 (16, u'Balzac', u'Honore de', u'moyen'),
 (18, u'Celine', u'Louis Ferdinand', u'gros'),
 (19, u'Ciceron', None, u'gros'),
 (20, u'Chateaubriand', u'Francois-Rene de', u'moyen'),
 (22, u'Cocteau', u'Jean', u'gros'),
 (24, u'Courrier', u'Paul Louis', u'moyen'),
 (26, u'Diderot', u'Denis', u'moyen'),
 (27, u'Dumas', u'Alexandre', u'gros'),
 (28, u'Dostoievsky', u'Fiodor', u'gros'),
 (31, u'Dante', None, u'gros'),
 (32, u'Eluard', u'Paul', u'moyen'),
 (33, u'Flaubert', u'Gustave', u'moyen'),
 (35, u'Fromentin', u'Eugene', u'gros'),
 (39, u'Montesquieu', None, u'gros'),
 (42, u'Goethe', u'Johann Wolfgang von', u'moyen'),
 (43, u'Musset', u'Alfred de', u'gros'),
 (46, u'Hugo', u'Victor', u'moyen'),
 (48, u'Hardy', u'Thomas', u'moyen'),
 (50, u'Lautreamont', None, u'moyen'),
 (53, u'Lamartine', u'Alphonse de', u'gros'),
 (55, u'Michelet', u'Jules', u'gros'),
 (57, u'Marx', u'Karl', u'gros'),
 (58, u'Marivaux', u'Pierre Carlet de Chamblain de', u'gros'),
 (60, u'Ovide', None, u'gros'),
 (62, u'Proust', u'Marcel', u'moyen'),
 (64, u'Platon', None, u'gros'),
 (66, u'Perec', u'Georges', u'gros'),
 (68, u'Prevert', u'Jacques', u'gros'),
 (70, u'Paulhan', u'Jean', u'gros'),
 (72, u'Rabelais', u'Francois', u'gros'),
 (73, u'Poe', u'Allan Edgard', u'moyen'),
 (75, u'Ronsard', u'Pierre', u'moyen'),
 (76, u'Racine', u'Jean', u'gros'),
 (78, u'Sandier', u'Jules', u'moyen'),
 (79, u'Sade', u'Donatien Alphonse Fran\xbf\xbfs de', u'gros'),
 (81, u'Sartre', u'Jean-Paul', u'gros'),
 (83, u'Sophocle', None, u'gros'),
 (84, u'Sue', u'Eugene', u'moyen'),
 (86, u'Saint Exupery', u'Antoine de', u'gros'),
 (87, u'Segur', u'Comtesse de', u'gros'),
 (91, u'Verlaine', u'Paul', u'gros'),
 (93, u'Virgile', None, u'gros'),
 (94, u'Zola', u'Emile', u'moyen'),
 (95, u'Zorn', u'Fritz', u'moyen'),
 (98, u'Maturin', u'Robert', u'gros'),
 (99, u'Cesar', u'Jules', u'gros')]

#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [8]:
%%sql
DROP VIEW IF EXISTS buveurs_asec;
CREATE VIEW buveurs_asec AS
SELECT DISTINCT a.NB, NOM, PRENOM, TYPE
FROM acheteurs as a
EXCEPT
SELECT DISTINCT a1.NB, NOM, PRENOM, TYPE
FROM acheteurs as a1
JOIN achats ON a1.NB=achats.NB;



 * sqlite:///wine.db
Done.
Done.


[]

In [9]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY NB;

 * sqlite:///wine.db
Done.


[(11, u'Breton', u'Andre', u'petit'),
 (13, u'Barthes', u'Roland', u'moyen'),
 (16, u'Balzac', u'Honore de', u'moyen'),
 (18, u'Celine', u'Louis Ferdinand', u'gros'),
 (20, u'Chateaubriand', u'Francois-Rene de', u'moyen'),
 (21, u'Corbiere', u'Tristan', u'petit'),
 (23, u'Corneille', u'Pierre', u'petit'),
 (25, u'Char', u'Rene', u'petit'),
 (27, u'Dumas', u'Alexandre', u'gros'),
 (29, u'Fournier', u'Alain', u'petit'),
 (32, u'Eluard', u'Paul', u'moyen'),
 (34, u'Mauriac', u'Francois', u'petit'),
 (35, u'Fromentin', u'Eugene', u'gros'),
 (36, u'Maurois', u'Andre', u'petit'),
 (37, u'Freud', u'Sigmund', u'petit'),
 (39, u'Montesquieu', None, u'gros'),
 (42, u'Goethe', u'Johann Wolfgang von', u'moyen'),
 (43, u'Musset', u'Alfred de', u'gros'),
 (46, u'Hugo', u'Victor', u'moyen'),
 (47, u'Gary', u'Romain', u'petit'),
 (51, u'Lafargue', u'Paul', u'petit'),
 (53, u'Lamartine', u'Alphonse de', u'gros'),
 (54, u'Aurele', u'Marc', u'petit'),
 (55, u'Michelet', u'Jules', u'gros'),
 (56, u'Nerval', u'Gerard de', u'petit'),
 (57, u'Marx', u'Karl', u'gros'),
 (59, u'Nietzsche', u'Friedrich', u'petit'),
 (63, u'Maupassant', u'Guy de', u'petit'),
 (64, u'Platon', None, u'gros'),
 (65, u'Pascal', u'Blaise', u'petit'),
 (66, u'Perec', u'Georges', u'gros'),
 (67, u'Nin', u'Anais', u'petit'),
 (68, u'Prevert', u'Jacques', u'gros'),
 (69, u'Queneau', u'Raymond', u'petit'),
 (70, u'Paulhan', u'Jean', u'gros'),
 (71, u'Radiguet', u'Raymond', u'petit'),
 (72, u'Rabelais', u'Francois', u'gros'),
 (73, u'Poe', u'Allan Edgard', u'moyen'),
 (74, u'Rimbaud', u'Arthur', u'petit'),
 (76, u'Racine', u'Jean', u'gros'),
 (77, u'Rousseau', u'Jean-Jacques', u'petit'),
 (78, u'Sandier', u'Jules', u'moyen'),
 (79, u'Sade', u'Donatien Alphonse Fran\xbf\xbfs de', u'gros'),
 (81, u'Sartre', u'Jean-Paul', u'gros'),
 (83, u'Sophocle', None, u'gros'),
 (84, u'Sue', u'Eugene', u'moyen'),
 (85, u'Segalen', u'Victor', u'petit'),
 (86, u'Saint Exupery', u'Antoine de', u'gros'),
 (87, u'Segur', u'Comtesse de', u'gros'),
 (88, u'Shakespeare', u'William', u'petit'),
 (89, u'Stendhal', None, u'petit'),
 (90, u'Vian', u'Boris', u'petit'),
 (91, u'Verlaine', u'Paul', u'gros'),
 (93, u'Virgile', None, u'gros'),
 (95, u'Zorn', u'Fritz', u'moyen'),
 (98, u'Maturin', u'Robert', u'gros'),
 (99, u'Cesar', u'Jules', u'gros')]

#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [10]:
%%sql
DROP VIEW IF EXISTS buveurs_achats;
CREATE VIEW buveurs_achats AS
SELECT a1.NB, NOM, PRENOM, TYPE 
FROM acheteurs as a1
EXCEPT
SELECT a2.NB, NOM, PRENOM, TYPE 
FROM buveurs_asec as a2;



 * sqlite:///wine.db
Done.
Done.


[]

In [11]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

 * sqlite:///wine.db
Done.


[(1, u'Aristote', None, u'petit'),
 (2, u'Artaud', u'Antonin', u'moyen'),
 (3, u'Aron', u'Raymond', u'gros'),
 (4, u'Apollinaire', u'Guillaume', u'moyen'),
 (5, u'Audiberti', u'Jacques', u'petit'),
 (6, u'Arrabal', u'Fernando', u'gros'),
 (7, u'Anouilh', u'Jean', u'moyen'),
 (8, u'Aragon', u'Louis', u'gros'),
 (9, u'Ajar', u'Emile', u'petit'),
 (10, u'Andersen', u'Yann', u'gros'),
 (12, u'Bataille', u'Georges', u'moyen'),
 (14, u'Bory', u'Jean Louis', u'gros'),
 (15, u'Baudelaire', u'Charles', u'petit'),
 (17, u'Camus', u'Albert', u'petit'),
 (19, u'Ciceron', None, u'gros'),
 (22, u'Cocteau', u'Jean', u'gros'),
 (24, u'Courrier', u'Paul Louis', u'moyen'),
 (26, u'Diderot', u'Denis', u'moyen'),
 (28, u'Dostoievsky', u'Fiodor', u'gros'),
 (30, u'La Fontaine', u'Jean de', u'petit'),
 (31, u'Dante', None, u'gros'),
 (33, u'Flaubert', u'Gustave', u'moyen'),
 (38, u'Montaigne', u'Michel de', u'petit'),
 (40, u'Moliere', None, u'petit'),
 (41, u'Gauthier', u'Theophile', u'petit'),
 (44, u'Gide', u'Andre', u'petit'),
 (45, u'Giono', u'Jean', u'petit'),
 (48, u'Hardy', u'Thomas', u'moyen'),
 (49, u'Kafka', u'Franz', u'petit'),
 (50, u'Lautreamont', None, u'moyen'),
 (52, u'Carroll', u'Lewis', u'petit'),
 (58, u'Marivaux', u'Pierre Carlet de Chamblain de', u'gros'),
 (60, u'Ovide', None, u'gros'),
 (61, u'Mallarme', u'Stephane', u'petit'),
 (62, u'Proust', u'Marcel', u'moyen'),
 (75, u'Ronsard', u'Pierre', u'moyen'),
 (80, u'Sand', u'Georges', u'petit'),
 (82, u'Roussel', u'Rene', u'petit'),
 (92, u'Verne', u'Jules', u'petit'),
 (94, u'Zola', u'Emile', u'moyen'),
 (96, u'Wilde', u'Oscar', u'petit'),
 (97, u'Vigny', u'Alfred de', u'petit'),
 (100, u'Woolf', u'Virginia', u'petit')]

#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [12]:
%%sql
DROP VIEW IF EXISTS q83pl;
CREATE VIEW q83pl AS
SELECT LIEU, CRU, SUM(QTE) as QTE_BUE 
FROM achats JOIN vins2 on vins2.NV=achats.NV
WHERE achats.DATES between '1983-01-01' and '1983-12-31'
GROUP BY LIEU, CRU



 * sqlite:///wine.db
Done.
Done.


[]

In [13]:
# Test
%sql SELECT * FROM q83pl;

 * sqlite:///wine.db
Done.


[(u'CAEN', u'Seyssel', 3),
 (u'LILLE', u'Pommard', 5),
 (u'LYON', u'Beaujolais Villages', 10),
 (u'LYON', u'Julienas', 2),
 (u'PARIS', u'Beaujolais Primeur', 4),
 (u'PARIS', u'Coteaux du Tricastin', 1),
 (u'PARIS', u'Pouilly Vinzelles', 3),
 (u'RENNES', u'Mercurey', 1),
 (u'ROCQUENCOURT', u'Beaujolais Villages', 260),
 (u'ROCQUENCOURT', u'Saint Amour', 80)]

#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [14]:
%%sql 
DROP TABLE IF EXISTS RBB;
CREATE TABLE RBB AS
SELECT *
FROM bons_buveurs;



 * sqlite:///wine.db
Done.
Done.


[]

In [15]:
# Test
%sql SELECT * FROM RBB ORDER BY NB;

 * sqlite:///wine.db
Done.


[(2, u'Artaud', u'Antonin', u'moyen'),
 (3, u'Aron', u'Raymond', u'gros'),
 (4, u'Apollinaire', u'Guillaume', u'moyen'),
 (6, u'Arrabal', u'Fernando', u'gros'),
 (7, u'Anouilh', u'Jean', u'moyen'),
 (8, u'Aragon', u'Louis', u'gros'),
 (10, u'Andersen', u'Yann', u'gros'),
 (12, u'Bataille', u'Georges', u'moyen'),
 (13, u'Barthes', u'Roland', u'moyen'),
 (14, u'Bory', u'Jean Louis', u'gros'),
 (16, u'Balzac', u'Honore de', u'moyen'),
 (18, u'Celine', u'Louis Ferdinand', u'gros'),
 (19, u'Ciceron', None, u'gros'),
 (20, u'Chateaubriand', u'Francois-Rene de', u'moyen'),
 (22, u'Cocteau', u'Jean', u'gros'),
 (24, u'Courrier', u'Paul Louis', u'moyen'),
 (26, u'Diderot', u'Denis', u'moyen'),
 (27, u'Dumas', u'Alexandre', u'gros'),
 (28, u'Dostoievsky', u'Fiodor', u'gros'),
 (31, u'Dante', None, u'gros'),
 (32, u'Eluard', u'Paul', u'moyen'),
 (33, u'Flaubert', u'Gustave', u'moyen'),
 (35, u'Fromentin', u'Eugene', u'gros'),
 (39, u'Montesquieu', None, u'gros'),
 (42, u'Goethe', u'Johann Wolfgang von', u'moyen'),
 (43, u'Musset', u'Alfred de', u'gros'),
 (46, u'Hugo', u'Victor', u'moyen'),
 (48, u'Hardy', u'Thomas', u'moyen'),
 (50, u'Lautreamont', None, u'moyen'),
 (53, u'Lamartine', u'Alphonse de', u'gros'),
 (55, u'Michelet', u'Jules', u'gros'),
 (57, u'Marx', u'Karl', u'gros'),
 (58, u'Marivaux', u'Pierre Carlet de Chamblain de', u'gros'),
 (60, u'Ovide', None, u'gros'),
 (62, u'Proust', u'Marcel', u'moyen'),
 (64, u'Platon', None, u'gros'),
 (66, u'Perec', u'Georges', u'gros'),
 (68, u'Prevert', u'Jacques', u'gros'),
 (70, u'Paulhan', u'Jean', u'gros'),
 (72, u'Rabelais', u'Francois', u'gros'),
 (73, u'Poe', u'Allan Edgard', u'moyen'),
 (75, u'Ronsard', u'Pierre', u'moyen'),
 (76, u'Racine', u'Jean', u'gros'),
 (78, u'Sandier', u'Jules', u'moyen'),
 (79, u'Sade', u'Donatien Alphonse Fran\xbf\xbfs de', u'gros'),
 (81, u'Sartre', u'Jean-Paul', u'gros'),
 (83, u'Sophocle', None, u'gros'),
 (84, u'Sue', u'Eugene', u'moyen'),
 (86, u'Saint Exupery', u'Antoine de', u'gros'),
 (87, u'Segur', u'Comtesse de', u'gros'),
 (91, u'Verlaine', u'Paul', u'gros'),
 (93, u'Virgile', None, u'gros'),
 (94, u'Zola', u'Emile', u'moyen'),
 (95, u'Zorn', u'Fritz', u'moyen'),
 (98, u'Maturin', u'Robert', u'gros'),
 (99, u'Cesar', u'Jules', u'gros')]

#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may differ from the one in your table)

In [16]:
%%sql
DROP VIEW IF EXISTS over100;
CREATE VIEW over100 AS
SELECT acheteurs.NB, NOM, PRENOM, TYPE, SUM(QTE)
FROM acheteurs JOIN achats ON acheteurs.NB=achats.NB GROUP BY achats.NB HAVING SUM(QTE)>100;
SELECT * FROM over100;

 * sqlite:///wine.db
Done.
Done.
Done.


[(2, u'Artaud', u'Antonin', u'moyen', 583),
 (5, u'Audiberti', u'Jacques', u'petit', 113),
 (8, u'Aragon', u'Louis', u'gros', 132),
 (9, u'Ajar', u'Emile', u'petit', 140),
 (44, u'Gide', u'Andre', u'petit', 171)]

Update instances

In [17]:
%%sql
UPDATE acheteurs
SET TYPE = "gros"
WHERE acheteurs.NB in (SELECT over100.NB FROM over100);

 * sqlite:///wine.db
5 rows affected.


[]

#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

In [18]:
%%sql 
SELECT * FROM bons_buveurs;

 * sqlite:///wine.db
Done.


[(13, u'Barthes', u'Roland', u'moyen'),
 (16, u'Balzac', u'Honore de', u'moyen'),
 (18, u'Celine', u'Louis Ferdinand', u'gros'),
 (20, u'Chateaubriand', u'Francois-Rene de', u'moyen'),
 (27, u'Dumas', u'Alexandre', u'gros'),
 (32, u'Eluard', u'Paul', u'moyen'),
 (35, u'Fromentin', u'Eugene', u'gros'),
 (39, u'Montesquieu', None, u'gros'),
 (42, u'Goethe', u'Johann Wolfgang von', u'moyen'),
 (43, u'Musset', u'Alfred de', u'gros'),
 (46, u'Hugo', u'Victor', u'moyen'),
 (53, u'Lamartine', u'Alphonse de', u'gros'),
 (55, u'Michelet', u'Jules', u'gros'),
 (57, u'Marx', u'Karl', u'gros'),
 (64, u'Platon', None, u'gros'),
 (66, u'Perec', u'Georges', u'gros'),
 (68, u'Prevert', u'Jacques', u'gros'),
 (70, u'Paulhan', u'Jean', u'gros'),
 (72, u'Rabelais', u'Francois', u'gros'),
 (73, u'Poe', u'Allan Edgard', u'moyen'),
 (76, u'Racine', u'Jean', u'gros'),
 (78, u'Sandier', u'Jules', u'moyen'),
 (79, u'Sade', u'Donatien Alphonse Fran\xbf\xbfs de', u'gros'),
 (81, u'Sartre', u'Jean-Paul', u'gros'),
 (83, u'Sophocle', None, u'gros'),
 (84, u'Sue', u'Eugene', u'moyen'),
 (86, u'Saint Exupery', u'Antoine de', u'gros'),
 (87, u'Segur', u'Comtesse de', u'gros'),
 (91, u'Verlaine', u'Paul', u'gros'),
 (93, u'Virgile', None, u'gros'),
 (95, u'Zorn', u'Fritz', u'moyen'),
 (98, u'Maturin', u'Robert', u'gros'),
 (99, u'Cesar', u'Jules', u'gros'),
 (2, u'Artaud', u'Antonin', u'gros'),
 (44, u'Gide', u'Andre', u'gros'),
 (48, u'Hardy', u'Thomas', u'moyen'),
 (7, u'Anouilh', u'Jean', u'moyen'),
 (8, u'Aragon', u'Louis', u'gros'),
 (50, u'Lautreamont', None, u'moyen'),
 (3, u'Aron', u'Raymond', u'gros'),
 (4, u'Apollinaire', u'Guillaume', u'moyen'),
 (58, u'Marivaux', u'Pierre Carlet de Chamblain de', u'gros'),
 (9, u'Ajar', u'Emile', u'gros'),
 (60, u'Ovide', None, u'gros'),
 (5, u'Audiberti', u'Jacques', u'gros'),
 (10, u'Andersen', u'Yann', u'gros'),
 (6, u'Arrabal', u'Fernando', u'gros'),
 (62, u'Proust', u'Marcel', u'moyen'),
 (75, u'Ronsard', u'Pierre', u'moyen'),
 (26, u'Diderot', u'Denis', u'moyen'),
 (28, u'Dostoievsky', u'Fiodor', u'gros'),
 (22, u'Cocteau', u'Jean', u'gros'),
 (24, u'Courrier', u'Paul Louis', u'moyen'),
 (12, u'Bataille', u'Georges', u'moyen'),
 (14, u'Bory', u'Jean Louis', u'gros'),
 (19, u'Ciceron', None, u'gros'),
 (33, u'Flaubert', u'Gustave', u'moyen'),
 (31, u'Dante', None, u'gros'),
 (94, u'Zola', u'Emile', u'moyen')]

#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [19]:
%%sql 
DROP TABLE IF EXISTS RBA;
CREATE TABLE RBA AS
SELECT *
FROM buveurs_asec;




 * sqlite:///wine.db
Done.
Done.


[]

In [20]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


[(11, u'Breton', u'Andre', u'petit'),
 (13, u'Barthes', u'Roland', u'moyen'),
 (16, u'Balzac', u'Honore de', u'moyen'),
 (18, u'Celine', u'Louis Ferdinand', u'gros'),
 (20, u'Chateaubriand', u'Francois-Rene de', u'moyen'),
 (21, u'Corbiere', u'Tristan', u'petit'),
 (23, u'Corneille', u'Pierre', u'petit'),
 (25, u'Char', u'Rene', u'petit'),
 (27, u'Dumas', u'Alexandre', u'gros'),
 (29, u'Fournier', u'Alain', u'petit'),
 (32, u'Eluard', u'Paul', u'moyen'),
 (34, u'Mauriac', u'Francois', u'petit'),
 (35, u'Fromentin', u'Eugene', u'gros'),
 (36, u'Maurois', u'Andre', u'petit'),
 (37, u'Freud', u'Sigmund', u'petit'),
 (39, u'Montesquieu', None, u'gros'),
 (42, u'Goethe', u'Johann Wolfgang von', u'moyen'),
 (43, u'Musset', u'Alfred de', u'gros'),
 (46, u'Hugo', u'Victor', u'moyen'),
 (47, u'Gary', u'Romain', u'petit'),
 (51, u'Lafargue', u'Paul', u'petit'),
 (53, u'Lamartine', u'Alphonse de', u'gros'),
 (54, u'Aurele', u'Marc', u'petit'),
 (55, u'Michelet', u'Jules', u'gros'),
 (56, u'Nerval', u'Gerard de', u'petit'),
 (57, u'Marx', u'Karl', u'gros'),
 (59, u'Nietzsche', u'Friedrich', u'petit'),
 (63, u'Maupassant', u'Guy de', u'petit'),
 (64, u'Platon', None, u'gros'),
 (65, u'Pascal', u'Blaise', u'petit'),
 (66, u'Perec', u'Georges', u'gros'),
 (67, u'Nin', u'Anais', u'petit'),
 (68, u'Prevert', u'Jacques', u'gros'),
 (69, u'Queneau', u'Raymond', u'petit'),
 (70, u'Paulhan', u'Jean', u'gros'),
 (71, u'Radiguet', u'Raymond', u'petit'),
 (72, u'Rabelais', u'Francois', u'gros'),
 (73, u'Poe', u'Allan Edgard', u'moyen'),
 (74, u'Rimbaud', u'Arthur', u'petit'),
 (76, u'Racine', u'Jean', u'gros'),
 (77, u'Rousseau', u'Jean-Jacques', u'petit'),
 (78, u'Sandier', u'Jules', u'moyen'),
 (79, u'Sade', u'Donatien Alphonse Fran\xbf\xbfs de', u'gros'),
 (81, u'Sartre', u'Jean-Paul', u'gros'),
 (83, u'Sophocle', None, u'gros'),
 (84, u'Sue', u'Eugene', u'moyen'),
 (85, u'Segalen', u'Victor', u'petit'),
 (86, u'Saint Exupery', u'Antoine de', u'gros'),
 (87, u'Segur', u'Comtesse de', u'gros'),
 (88, u'Shakespeare', u'William', u'petit'),
 (89, u'Stendhal', None, u'petit'),
 (90, u'Vian', u'Boris', u'petit'),
 (91, u'Verlaine', u'Paul', u'gros'),
 (93, u'Virgile', None, u'gros'),
 (95, u'Zorn', u'Fritz', u'moyen'),
 (98, u'Maturin', u'Robert', u'gros'),
 (99, u'Cesar', u'Jules', u'gros')]

#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [21]:
%%sql
INSERT INTO RBA  
VALUES (101, "Medvedev", "Anton","petit");

 * sqlite:///wine.db
1 rows affected.


[]

In [22]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


[(11, u'Breton', u'Andre', u'petit'),
 (13, u'Barthes', u'Roland', u'moyen'),
 (16, u'Balzac', u'Honore de', u'moyen'),
 (18, u'Celine', u'Louis Ferdinand', u'gros'),
 (20, u'Chateaubriand', u'Francois-Rene de', u'moyen'),
 (21, u'Corbiere', u'Tristan', u'petit'),
 (23, u'Corneille', u'Pierre', u'petit'),
 (25, u'Char', u'Rene', u'petit'),
 (27, u'Dumas', u'Alexandre', u'gros'),
 (29, u'Fournier', u'Alain', u'petit'),
 (32, u'Eluard', u'Paul', u'moyen'),
 (34, u'Mauriac', u'Francois', u'petit'),
 (35, u'Fromentin', u'Eugene', u'gros'),
 (36, u'Maurois', u'Andre', u'petit'),
 (37, u'Freud', u'Sigmund', u'petit'),
 (39, u'Montesquieu', None, u'gros'),
 (42, u'Goethe', u'Johann Wolfgang von', u'moyen'),
 (43, u'Musset', u'Alfred de', u'gros'),
 (46, u'Hugo', u'Victor', u'moyen'),
 (47, u'Gary', u'Romain', u'petit'),
 (51, u'Lafargue', u'Paul', u'petit'),
 (53, u'Lamartine', u'Alphonse de', u'gros'),
 (54, u'Aurele', u'Marc', u'petit'),
 (55, u'Michelet', u'Jules', u'gros'),
 (56, u'Nerval', u'Gerard de', u'petit'),
 (57, u'Marx', u'Karl', u'gros'),
 (59, u'Nietzsche', u'Friedrich', u'petit'),
 (63, u'Maupassant', u'Guy de', u'petit'),
 (64, u'Platon', None, u'gros'),
 (65, u'Pascal', u'Blaise', u'petit'),
 (66, u'Perec', u'Georges', u'gros'),
 (67, u'Nin', u'Anais', u'petit'),
 (68, u'Prevert', u'Jacques', u'gros'),
 (69, u'Queneau', u'Raymond', u'petit'),
 (70, u'Paulhan', u'Jean', u'gros'),
 (71, u'Radiguet', u'Raymond', u'petit'),
 (72, u'Rabelais', u'Francois', u'gros'),
 (73, u'Poe', u'Allan Edgard', u'moyen'),
 (74, u'Rimbaud', u'Arthur', u'petit'),
 (76, u'Racine', u'Jean', u'gros'),
 (77, u'Rousseau', u'Jean-Jacques', u'petit'),
 (78, u'Sandier', u'Jules', u'moyen'),
 (79, u'Sade', u'Donatien Alphonse Fran\xbf\xbfs de', u'gros'),
 (81, u'Sartre', u'Jean-Paul', u'gros'),
 (83, u'Sophocle', None, u'gros'),
 (84, u'Sue', u'Eugene', u'moyen'),
 (85, u'Segalen', u'Victor', u'petit'),
 (86, u'Saint Exupery', u'Antoine de', u'gros'),
 (87, u'Segur', u'Comtesse de', u'gros'),
 (88, u'Shakespeare', u'William', u'petit'),
 (89, u'Stendhal', None, u'petit'),
 (90, u'Vian', u'Boris', u'petit'),
 (91, u'Verlaine', u'Paul', u'gros'),
 (93, u'Virgile', None, u'gros'),
 (95, u'Zorn', u'Fritz', u'moyen'),
 (98, u'Maturin', u'Robert', u'gros'),
 (99, u'Cesar', u'Jules', u'gros'),
 (101, u'Medvedev', u'Anton', u'petit')]

#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain

In [23]:
%%sql 
SELECT * FROM buveurs_asec;

 * sqlite:///wine.db
Done.


[(11, u'Breton', u'Andre', u'petit'),
 (13, u'Barthes', u'Roland', u'moyen'),
 (16, u'Balzac', u'Honore de', u'moyen'),
 (18, u'Celine', u'Louis Ferdinand', u'gros'),
 (20, u'Chateaubriand', u'Francois-Rene de', u'moyen'),
 (21, u'Corbiere', u'Tristan', u'petit'),
 (23, u'Corneille', u'Pierre', u'petit'),
 (25, u'Char', u'Rene', u'petit'),
 (27, u'Dumas', u'Alexandre', u'gros'),
 (29, u'Fournier', u'Alain', u'petit'),
 (32, u'Eluard', u'Paul', u'moyen'),
 (34, u'Mauriac', u'Francois', u'petit'),
 (35, u'Fromentin', u'Eugene', u'gros'),
 (36, u'Maurois', u'Andre', u'petit'),
 (37, u'Freud', u'Sigmund', u'petit'),
 (39, u'Montesquieu', None, u'gros'),
 (42, u'Goethe', u'Johann Wolfgang von', u'moyen'),
 (43, u'Musset', u'Alfred de', u'gros'),
 (46, u'Hugo', u'Victor', u'moyen'),
 (47, u'Gary', u'Romain', u'petit'),
 (51, u'Lafargue', u'Paul', u'petit'),
 (53, u'Lamartine', u'Alphonse de', u'gros'),
 (54, u'Aurele', u'Marc', u'petit'),
 (55, u'Michelet', u'Jules', u'gros'),
 (56, u'Nerval', u'Gerard de', u'petit'),
 (57, u'Marx', u'Karl', u'gros'),
 (59, u'Nietzsche', u'Friedrich', u'petit'),
 (63, u'Maupassant', u'Guy de', u'petit'),
 (64, u'Platon', None, u'gros'),
 (65, u'Pascal', u'Blaise', u'petit'),
 (66, u'Perec', u'Georges', u'gros'),
 (67, u'Nin', u'Anais', u'petit'),
 (68, u'Prevert', u'Jacques', u'gros'),
 (69, u'Queneau', u'Raymond', u'petit'),
 (70, u'Paulhan', u'Jean', u'gros'),
 (71, u'Radiguet', u'Raymond', u'petit'),
 (72, u'Rabelais', u'Francois', u'gros'),
 (73, u'Poe', u'Allan Edgard', u'moyen'),
 (74, u'Rimbaud', u'Arthur', u'petit'),
 (76, u'Racine', u'Jean', u'gros'),
 (77, u'Rousseau', u'Jean-Jacques', u'petit'),
 (78, u'Sandier', u'Jules', u'moyen'),
 (79, u'Sade', u'Donatien Alphonse Fran\xbf\xbfs de', u'gros'),
 (81, u'Sartre', u'Jean-Paul', u'gros'),
 (83, u'Sophocle', None, u'gros'),
 (84, u'Sue', u'Eugene', u'moyen'),
 (85, u'Segalen', u'Victor', u'petit'),
 (86, u'Saint Exupery', u'Antoine de', u'gros'),
 (87, u'Segur', u'Comtesse de', u'gros'),
 (88, u'Shakespeare', u'William', u'petit'),
 (89, u'Stendhal', None, u'petit'),
 (90, u'Vian', u'Boris', u'petit'),
 (91, u'Verlaine', u'Paul', u'gros'),
 (93, u'Virgile', None, u'gros'),
 (95, u'Zorn', u'Fritz', u'moyen'),
 (98, u'Maturin', u'Robert', u'gros'),
 (99, u'Cesar', u'Jules', u'gros')]

___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

<img src="q1.jpg">

#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.

<img src="q2.jpg">